In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
!pip install transformers
!pip install simpletransformers

In [3]:
# install simpleT5
!pip install simplet5

In [4]:
import numpy as np
import pandas as pd
import os, json, gc, re, random
from tqdm.notebook import tqdm
# from tqdm import tqdm
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [5]:
import torch, transformers, tokenizers
torch.__version__, transformers.__version__, tokenizers.__version__

('1.9.0+cu102', '4.6.1', '0.10.3')

In [6]:
cd drive/My Drive/Colab Notebooks/experiments

/content/drive/My Drive/Colab Notebooks/experiments


##TroFi-X - Abstract Summarization (Noun+Verb)

In [7]:
data_file = pd.read_csv("data/trofix.csv")

In [8]:
data_file = data_file.rename(columns={'sentence': 'source_text'}) #abstract - sentence
data_file['target_text'] = data_file[['arg1', 'arg2', 'verb']].agg(' '.join, axis=1) #title - metaphorical words
#data_file['target_text'] = data_file['verb'] #title - metaphorical words

In [9]:
papers = data_file[["source_text", "target_text"]]
papers.head()

,source_text,target_text
0,Triple mileage has struck another blow to the ...,mileage struck blow
1,U.S. officials said evidence suggests that a J...,terrorist attack target
2,"Some police forces , for example , have steppe...",forces stepped use
3,"Every day his troops gather under the green , ...",day pour stream
4,He says manufacturers are increasingly rolling...,manufacturers rolling products


In [10]:
# Load test data - TroFi
import pandas as pd
# Use a subset for quick experiments
#data = data[:10000]

from sklearn.model_selection import train_test_split as tts
import pandas as pd

# Split to train, val and test
train_df, test_data = tts(data_file[["source_text", "target_text"]], random_state=42, test_size=0.1)
train_df, val = tts(train_df, random_state=42, test_size=test_data.shape[0])

In [11]:
%%time

from simpletransformers.seq2seq import Seq2SeqModel

test_df = pd.read_csv("stockholm/bert_code/trofix_bert_subs/trofix_all_lit.csv")
test_df = test_df.rename(columns={'sentence': 'source_text'}) #abstract - sentence
test_df['target_text'] = test_df[['arg1', 'arg2', 'verb']].agg(' '.join, axis=1) #title - metaphorical words

CPU times: user 2.29 s, sys: 399 ms, total: 2.69 s
Wall time: 4.31 s


In [12]:
# import
from simplet5 import SimpleT5

# instatntiate
model = SimpleT5()

# load
model.from_pretrained("t5","t5-base")

# train
model.train(train_df=train_df, eval_df=test_df, source_max_token_len=512, target_max_token_len=128, max_epochs=5, batch_size=8, use_gpu=True)

INFO:pytorch_lightning.utilities.seed:Global seed set to 42
INFO:pytorch_lightning.utilities.distributed:GPU available: True, used: True
INFO:pytorch_lightning.utilities.distributed:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.core.lightning:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [13]:
!ls outputs/

checkpoint-10000		    SimpleT5-epoch-2-train-loss-0.2453
checkpoint-2000			    SimpleT5-epoch-2-train-loss-0.2695
checkpoint-4000			    SimpleT5-epoch-2-train-loss-0.2735
checkpoint-6000			    SimpleT5-epoch-2-train-loss-0.3045
checkpoint-8000			    SimpleT5-epoch-2-train-loss-0.3159
config.json			    SimpleT5-epoch-2-train-loss-0.3466
eval_results.txt		    SimpleT5-epoch-2-train-loss-0.3474
merges.txt			    SimpleT5-epoch-2-train-loss-0.3475
model_args.json			    SimpleT5-epoch-2-train-loss-0.787
pytorch_model.bin		    SimpleT5-epoch-3-train-loss-0.0384
SimpleT5-epoch-0-train-loss-0.8219  SimpleT5-epoch-3-train-loss-0.0431
SimpleT5-epoch-0-train-loss-0.858   SimpleT5-epoch-3-train-loss-0.0493
SimpleT5-epoch-0-train-loss-0.8869  SimpleT5-epoch-3-train-loss-0.0702
SimpleT5-epoch-0-train-loss-1.0937  SimpleT5-epoch-3-train-loss-0.1619
SimpleT5-epoch-0-train-loss-1.4472  SimpleT5-epoch-3-train-loss-0.1764
SimpleT5-epoch-0-train-loss-1.8205  SimpleT5-epoch-3-train-loss-0.1886
SimpleT5-epoc

In [14]:
# load a trained model
model.load_model("outputs/SimpleT5-epoch-4-train-loss-1.1577", use_gpu=True)

In [15]:
# let's see how it performerd:
sample_abstracts = test_df.sample(35)

for i, abstract in sample_abstracts.iterrows():
    print(f"===== Original Sentence =====")
    print(abstract['source_text'])
    summary= model.predict(abstract['source_text'])[0]
    print(f"\n===== Actual Metaphorical Tokens =====")
    print(f"{abstract['target_text']}")
    print(f"\n===== Predicted Metaphorical Tokens =====")
    print(f"{summary}")
    print("\n +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\n")

===== Original Sentence =====
But the report , published in today 's edition of Science , lends credence to a body of opinion that HIV-2 in some cases may not ravage T-4 cells -- immune sentries of the bloodstream -- as much as it harms cells of the brain or bowel , said Dr. Levy in an interview .

===== Actual Metaphorical Tokens =====
report lends credence

===== Predicted Metaphorical Tokens =====
it ravage cells

 +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

===== Original Sentence =====
My late father-in-law , Jean Le Gorre , once recounted an early experience in the French army during World War I where on bivouac he examined his field ration -- a piece of issue `` hardtack . ''

===== Actual Metaphorical Tokens =====
he examined ration

===== Predicted Metaphorical Tokens =====
he examined ration

 +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

===== Original Sentence =====
Baaron Pittenger , a long-time US